In [22]:
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from IPython.display import display
from PIL.Image import fromarray, Image
from torch import Tensor
from torch.nn.modules.loss import CrossEntropyLoss
from torch.optim.adam import Adam
from torch.utils.data import DataLoader
from torchvision.datasets.mnist import MNIST
from torchvision.transforms import ToTensor

In [14]:
train_data: MNIST = MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data: MNIST = MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [15]:
def display_img_with_idx(
    ds: MNIST,
    idx: int
) -> None:
    if idx >= len(ds) or idx < 0:
        raise ValueError("Index out of range")
    else:
        img: Tensor = ds.__getitem__(idx)[0]

        if img.is_cuda:
            img = img.cpu()

        img_arr: np.ndarray = img.numpy().transpose(1, 2, 0)
        if img_arr.max() <= 1:  # normalize
            img_arr = (img_arr * 255).astype(np.uint8)
        if img_arr.shape[2] == 1:  # grayscale
            img_arr = img_arr.squeeze(axis=2)

        img_pil: Image = fromarray(img_arr)
        display(img_pil)


for i in range(5):
    display_img_with_idx(train_data, i)

In [16]:
batch_size: int = 100

loaders: dict[str, DataLoader] = {
    "train": DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=3
    ),
    "test": DataLoader(
        test_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=3
    )
}

In [17]:
# Define the neural network.
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        
        return F.softmax(x)

In [18]:
device: str = torch.device(
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)
print(f"Using {device} device...")

model: CNN = CNN().to(device)
optimizer: Adam = Adam(model.parameters(), lr=0.001)
loss_fn: CrossEntropyLoss = CrossEntropyLoss()

Using cuda device...


In [19]:
def train(epoch: int) -> None:
    model.train()
    
    for batch_idx, (data, target) in enumerate(loaders["train"]):
        data: Tensor = data.to(device)
        target: Tensor = target.to(device)
            
        optimizer.zero_grad()
        
        output: Tensor = model(data)
        loss: Tensor = loss_fn(output, target)
        
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print(f"Train Epoch: {epoch} [{(100.0 * (batch_idx + 1) / len(loaders['train'])):.4f}%]\tLoss: {loss.item():.4f}")
            
            
def test() -> None:
    model.eval()
    
    test_loss: float = 0.0
    correct: int = 0
        
    with torch.no_grad():
        for data, target in loaders["test"]:
            data: Tensor = data.to(device)
            target: Tensor = target.to(device)
                
            output: Tensor = model(data)
            pred: Tensor = output.argmax(dim=1, keepdim=True)
            
            test_loss += loss_fn(output, target).item()
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(loaders["test"].dataset)
    print(f"\nTest Set:\n\tAverage Loss: {test_loss:.4f}\n\tAccuracy: {(100.0 * correct / len(loaders['test'].dataset)):.4f}%\n")

In [20]:
for epoch in range(1, 11):
    train(epoch)
    test() 

/tmp/ipykernel_34/1640568162.py:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


Train Epoch: 1 [0.1667%]	Loss: 2.3021
Train Epoch: 1 [16.8333%]	Loss: 1.8864
Train Epoch: 1 [33.5000%]	Loss: 1.7812
Train Epoch: 1 [50.1667%]	Loss: 1.7576
Train Epoch: 1 [66.8333%]	Loss: 1.6509
Train Epoch: 1 [83.5000%]	Loss: 1.6553

Test Set:
	Average Loss: 0.0153
	Accuracy: 93.3300%

Train Epoch: 2 [0.1667%]	Loss: 1.6162
Train Epoch: 2 [16.8333%]	Loss: 1.6136
Train Epoch: 2 [33.5000%]	Loss: 1.5785
Train Epoch: 2 [50.1667%]	Loss: 1.6362
Train Epoch: 2 [66.8333%]	Loss: 1.5853
Train Epoch: 2 [83.5000%]	Loss: 1.5512

Test Set:
	Average Loss: 0.0151
	Accuracy: 95.0100%

Train Epoch: 3 [0.1667%]	Loss: 1.5805
Train Epoch: 3 [16.8333%]	Loss: 1.5740
Train Epoch: 3 [33.5000%]	Loss: 1.5428
Train Epoch: 3 [50.1667%]	Loss: 1.5423
Train Epoch: 3 [66.8333%]	Loss: 1.5281
Train Epoch: 3 [83.5000%]	Loss: 1.5544

Test Set:
	Average Loss: 0.0150
	Accuracy: 95.8100%

Train Epoch: 4 [0.1667%]	Loss: 1.5620
Train Epoch: 4 [16.8333%]	Loss: 1.5499
Train Epoch: 4 [33.5000%]	Loss: 1.5644
Train Epoch: 4 [50.1667

In [60]:
model.eval()

idx: int = random.randint(0, len(test_data))

data: Tensor; target: int
data, target = test_data.__getitem__(idx)
data = data.unsqueeze(0).to(device)

output: Tensor = model(data)
pred: int = output.argmax(dim=1, keepdim=True).item()

print(f"Index: {idx}\nPrediction: {pred}")
display_img_with_idx(test_data, idx)

Index: 4761
Prediction: 9


/tmp/ipykernel_34/1640568162.py:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)
